In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

In [3]:
# Carregar dados de treinamento
def load_data(images_dir, labels_dir):
    img_files = os.listdir(images_dir)
    img_files.sort()
    label_files = os.listdir(labels_dir)
    label_files.sort()
    
    complete_images = []
    complete_class = []
    for i in range(len(img_files)):
        img = plt.imread(os.path.join(images_dir, img_files[i]))
        with open(os.path.join(labels_dir, label_files[i]), 'r') as file:
            r = file.readlines()
        bounding_boxes = []
        for j in r:
            j = j.split()
            bounding_boxes.append([int(j[0]), float(j[1]), float(j[2]), float(j[3]), float(j[4])])
        for box in bounding_boxes:
            image_height, image_width, _ = img.shape
            xmin, ymin, width, height = box[1:]
            xmin = int(xmin * image_width)
            ymin = int(ymin * image_height)
            width = int(width * image_width)
            height = int(height * image_height)
            complete_class.append(box[0])
            complete_images.append(img[ymin-height//2:ymin+height//2, xmin-width//2:xmin+width//2])
    
    pref_size = (128, 128)
    for i in range(len(complete_images)):
        complete_images[i] = cv2.resize(complete_images[i], pref_size)
    
    df = pd.DataFrame()
    df['Images'] = complete_images
    df['Class'] = complete_class
    df['Images'] /= 255.0  # Normalizar as imagens
    
    return df

In [4]:
# Carregar dados de treinamento e validação
train_data = load_data('fall_dataset/images/train',
                       'fall_dataset/labels/train')
X_train, X_val, y_train, y_val = train_test_split(train_data['Images'].values,
                                                  train_data['Class'].values,
                                                  test_size=0.2,
                                                  random_state=42)

In [6]:
# Carregar dados de teste
test_data = load_data('fall_dataset/images/val',
                      'fall_dataset/labels/val')
X_test, y_test = test_data['Images'].values, test_data['Class'].values


In [16]:
# Converter os arrays numpy em tensores
X_test_tensor = tf.convert_to_tensor(np.stack(X_test), dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)

In [17]:
# Definir a arquitetura da CNN
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
for layer in base_model.layers:
    layer.trainable = False

model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))  # Mudar para 1 se for binária

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
# Converter os arrays numpy em tensores
X_train_tensor = tf.convert_to_tensor(np.stack(X_train), dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_val_tensor = tf.convert_to_tensor(np.stack(X_val), dtype=tf.float32)
y_val_tensor = tf.convert_to_tensor(y_val, dtype=tf.float32)

# Treinar o modelo
history = model.fit(X_train_tensor, y_train_tensor, epochs=20, validation_data=(X_val_tensor, y_val_tensor))


Epoch 1/20
12/12 [==============================] - 38s 3s/step - loss: 1.3861 - accuracy: 0.5276 - val_loss: 0.9264 - val_accuracy: 0.5495
Epoch 2/20
12/12 [==============================] - 49s 4s/step - loss: 0.5660 - accuracy: 0.7790 - val_loss: 0.5520 - val_accuracy: 0.7802
Epoch 3/20
12/12 [==============================] - 37s 3s/step - loss: 0.3469 - accuracy: 0.8646 - val_loss: 0.5693 - val_accuracy: 0.7253
Epoch 4/20
12/12 [==============================] - 29s 2s/step - loss: 0.2062 - accuracy: 0.9227 - val_loss: 0.4919 - val_accuracy: 0.7363
Epoch 5/20
12/12 [==============================] - 31s 3s/step - loss: 0.1707 - accuracy: 0.9365 - val_loss: 0.4928 - val_accuracy: 0.7582
Epoch 6/20
12/12 [==============================] - 30s 3s/step - loss: 0.1256 - accuracy: 0.9641 - val_loss: 0.5246 - val_accuracy: 0.7473
Epoch 7/20
12/12 [==============================] - 30s 3s/step - loss: 0.0968 - accuracy: 0.9669 - val_loss: 0.4501 - val_accuracy: 0.7802
Epoch 8/20
12/12 [==

In [20]:
model.save('quedas.keras')


In [21]:
# Avaliar o modelo no conjunto de teste
model.evaluate(X_test_tensor, y_test_tensor)

4/4 [==============================] - 8s 2s/step - loss: 0.6348 - accuracy: 0.7719


[0.6348479390144348, 0.7719298005104065]